# Building the ML dataframe

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

In [3]:
# Set max number of columns to display; default 20
pd.options.display.max_columns = 80

In [4]:
# Directory where data files will be downloaded
cwd_path = Path.cwd()
data_path = cwd_path.parent.joinpath('data')
data_push_path = cwd_path.parent.joinpath('data_to_push')

#### Load in latest wind turbine data

In [5]:
# Read in latest turbine data
# Now only 30,642 after removing 3 turbines outside bounding box
df_turbines = pd.read_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation.pkl')
df_turbines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30642 entries, 0 to 30641
Data columns (total 30 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   EinheitMastrNummer                                30642 non-null  object        
 1   DatumLetzteAktualisierung                         30642 non-null  datetime64[ns]
 2   Bundesland                                        30642 non-null  object        
 3   Postleitzahl                                      30642 non-null  int64         
 4   Ort                                               30642 non-null  object        
 5   Laengengrad                                       30642 non-null  float64       
 6   Breitengrad                                       30642 non-null  float64       
 7   Registrierungsdatum                               30642 non-null  datetime64[ns]
 8   Inbetriebnahmedatum       

-----

## Begin building ML dataframe
- Start with one full year: 2022
    - Then bring in another year like 2021 and run through the same transformations and then concat along datetimeindex?
- Good resource on time-related feature engineering: https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html

In [23]:
# Year 2022
# Make time zone aware UTC?
datetime_index_utc = pd.date_range(start='2022-01-01', end='2022-12-31 23:59:59', freq='H', name='datetime_utc', tz='UTC')
datetime_index_utc

DatetimeIndex(['2022-01-01 00:00:00+00:00', '2022-01-01 01:00:00+00:00',
               '2022-01-01 02:00:00+00:00', '2022-01-01 03:00:00+00:00',
               '2022-01-01 04:00:00+00:00', '2022-01-01 05:00:00+00:00',
               '2022-01-01 06:00:00+00:00', '2022-01-01 07:00:00+00:00',
               '2022-01-01 08:00:00+00:00', '2022-01-01 09:00:00+00:00',
               ...
               '2022-12-31 14:00:00+00:00', '2022-12-31 15:00:00+00:00',
               '2022-12-31 16:00:00+00:00', '2022-12-31 17:00:00+00:00',
               '2022-12-31 18:00:00+00:00', '2022-12-31 19:00:00+00:00',
               '2022-12-31 20:00:00+00:00', '2022-12-31 21:00:00+00:00',
               '2022-12-31 22:00:00+00:00', '2022-12-31 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='datetime_utc', length=8760, freq='H')

In [24]:
# Create dataframe from datetimeindex, reset index and drop the duplicated datetime column
# datetime_index_utc.to_frame().reset_index(drop=True, inplace=True)

In [25]:
df = datetime_index_utc.to_frame().reset_index(drop=True)
df

,datetime_utc
0,2022-01-01 00:00:00+00:00
1,2022-01-01 01:00:00+00:00
2,2022-01-01 02:00:00+00:00
3,2022-01-01 03:00:00+00:00
4,2022-01-01 04:00:00+00:00
...,...
8755,2022-12-31 19:00:00+00:00
8756,2022-12-31 20:00:00+00:00
8757,2022-12-31 21:00:00+00:00
8758,2022-12-31 22:00:00+00:00


In [30]:
# Extract properties
df['hour'] = df['datetime_utc'].dt.hour
# 0-6 (Monday-Sunday)
df['day_of_week'] = df['datetime_utc'].dt.dayofweek
df['day_of_month'] = df['datetime_utc'].dt.day
df['month_number'] = df['datetime_utc'].dt.month

# Function to get meteorological season based on month number (1-12)
# Should I just go ahead and encode the discrete numerical value?
def get_meteorological_season(month):
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'autumn'
    else:
        return 'winter'
    
df['meteorological_season'] = df['month_number'].apply(get_meteorological_season)

In [31]:
df

,datetime_utc,hour,day_of_week,day_of_month,month_number,meteorological_season
0,2022-01-01 00:00:00+00:00,0,5,1,1,winter
1,2022-01-01 01:00:00+00:00,1,5,1,1,winter
2,2022-01-01 02:00:00+00:00,2,5,1,1,winter
3,2022-01-01 03:00:00+00:00,3,5,1,1,winter
4,2022-01-01 04:00:00+00:00,4,5,1,1,winter
...,...,...,...,...,...,...
8755,2022-12-31 19:00:00+00:00,19,5,31,12,winter
8756,2022-12-31 20:00:00+00:00,20,5,31,12,winter
8757,2022-12-31 21:00:00+00:00,21,5,31,12,winter
8758,2022-12-31 22:00:00+00:00,22,5,31,12,winter
